In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from brainflow.data_filter import DataFilter, FilterTypes, DetrendOperations, WindowOperations
from sklearn.metrics import f1_score

In [2]:
class ToTensor(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.tensor(window.values).unsqueeze(0).to(torch.float32), 
                'labels': torch.tensor(labels).to(torch.float32)}#.values)}

class AbsoluteValue(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': torch.abs(window), 
                'labels': labels}

class Normalize(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        return {'window': F.normalize(window), 
                'labels': labels}

class MinNormalize(object):
    def __call__(self, sample):
        window, labels = sample['window'], sample['labels']
        min = torch.min(window)
        window = window - min
        window = window / torch.max(window)
        return {'window': (window), 
                'labels': labels}

class EEGDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.csv_file = pd.read_csv(csv_file, delimiter='\t', header=None)
        self.transform = transform
    
    def __len__(self):
        return len(self.csv_file)
    
    def __getitem__(self, idx):
        # if idx >= len(self.csv_file) / 1250:
        #     return

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        window = self.csv_file.iloc[idx*1250:idx*1250+1250, 7]
        labels = [self.csv_file.iloc[idx*1250, 32]]#:idx*1250+1250, 32] # remove brackets for timestep prediction
        sample = {'window': window, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [3]:
eeg_dataset = EEGDataset(csv_file='./fullData.csv', transform=transforms.Compose([ToTensor(),
                                                                                  AbsoluteValue(),
                                                                                  MinNormalize()]))

kuba_dataset = EEGDataset(csv_file='./data/kuba.csv', transform=transforms.Compose([ToTensor(),
                                                                                  AbsoluteValue(),
                                                                                  MinNormalize()]))

eeg_dataset = torch.utils.data.Subset(eeg_dataset, range(0, 104))
kuba_dataset = torch.utils.data.Subset(eeg_dataset, range(0, 48))

torch.manual_seed(50) # 80
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(eeg_dataset, [0.6, 0.2, 0.2])

print(len(train_dataset), len(val_dataset), len(test_dataset))

zerocount = 0
onecount = 0

for i in range(len(train_dataset)):
    if train_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

zerocount = 0
onecount = 0

for i in range(len(val_dataset)):
    if val_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

zerocount = 0
onecount = 0

for i in range(len(test_dataset)):
    if test_dataset[i]['labels'][0] == 1:
        onecount += 1
    else:
        zerocount += 1

print(onecount, zerocount)

63 21 20
33 30
10 11
9 11


In [4]:
num_train = len(train_dataset)
window_tensors = torch.empty(size=(num_train, 1, 1250))
label_tensors = torch.empty(size=(num_train, 1))

for i in range(num_train):
    window_tensors[i] = train_dataset[i]['window']
    label_tensors[i] = train_dataset[i]['labels']

torch.save(window_tensors, 'window_tensors.pt')
torch.save(label_tensors, 'label_tensors.pt')

window_tensors = torch.load('window_tensors.pt')
label_tensors = torch.load('label_tensors.pt')

print(window_tensors.shape)
print(label_tensors.shape)

torch.Size([63, 1, 1250])
torch.Size([63, 1])


/var/folders/jy/pdjxhl6d6q58q0b7w0sk_g_80000gn/T/ipykernel_2072/2291086987.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  window_tensors = torch.load('window_tensors.p

In [5]:
num_val = len(val_dataset)
val_window_tensors = torch.empty(size=(num_val, 1, 1250))
val_label_tensors = torch.empty(size=(num_val, 1))

for i in range(num_val):
    val_window_tensors[i] = val_dataset[i]['window']
    val_label_tensors[i] = val_dataset[i]['labels']

torch.save(val_window_tensors, 'val_window_tensors.pt')
torch.save(val_label_tensors, 'val_label_tensors.pt')

val_window_tensors = torch.load('val_window_tensors.pt')
val_label_tensors = torch.load('val_label_tensors.pt')

print(val_window_tensors.shape)
print(val_label_tensors.shape)

torch.Size([21, 1, 1250])
torch.Size([21, 1])


/var/folders/jy/pdjxhl6d6q58q0b7w0sk_g_80000gn/T/ipykernel_2072/622086804.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  val_window_tensors = torch.load('val_window_te

In [ ]:
class Model(nn.Module):
    def __init__(self, num_classes=3):
        super(Model, self).__init__()
        
      
        self.conv1 = nn.Conv1d(1, 32, 7, padding=3)
        self.in1 = nn.InstanceNorm2d(32)  
        
        self.conv2 = nn.Conv1d(32, 64, 3, padding=1)
        self.in2 = nn.InstanceNorm2d(64)
        
        self.conv3 = nn.Conv1d(64, 128, 3, padding=1)
        self.in3 = nn.InstanceNorm2d(128)
        
        self.pool = nn.AvgPool1d(3)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)
        
    def forward(self, x):
        print("Shape check", x.shape)
        x = self.conv1(x)
        print("Shape check 2", x.shape)
        x = F.relu(self.in1(x))
        print("Shape check 3", x.shape)
        #x = F.relu(self.in1(self.conv1(x)))
        x = self.pool(x)
        print("Shape check 4", x.shape)
        
        x = F.relu(self.in2(self.conv2(x)))
        print("Shape check 5", x.shape)
        x = self.pool(x)
        print("Shape check 6", x.shape)
        
        x = F.relu(self.in3(self.conv3(x)))
        print("Shape check 7", x.shape)
        x = self.pool(x)
        print("Shape check 8", x.shape)
        
        x = self.global_pool(x)
        print("Shape check 9", x.shape)
        x = torch.flatten(x, start_dim =1)
        print("Shape check 10", x.shape)
        test = self.fc(x)
        print("Final shape", test.shape)
        return self.fc(x)

model = Model()
model.load_state_dict(torch.load('80devf1.pt', weights_only=True, map_location='cpu'))

for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

RuntimeError: Error(s) in loading state_dict for Model:
	size mismatch for fc.weight: copying a param with shape torch.Size([3, 128]) from checkpoint, the shape in current model is torch.Size([2, 128]).
	size mismatch for fc.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:
optimizer = torch.optim.SGD(model.parameters())
criterion = nn.BCELoss()

In [ ]:
# lossi = []

# model.train()
# for epoch in tqdm(range(10000)):
#     optimizer.zero_grad()
#     y_pred = model(train_dataset[0]['window'])

#     # if epoch == 0:
#     #     print(y_pred)

#     loss = criterion(y_pred, train_dataset[0]['labels'])
#     lossi.append(loss.item())

#     loss.backward()
#     optimizer.step()
    
#     # if epoch <= 1:
#     #     for p in model.parameters():
#     #         print(p.grad)
    

# plt.plot(lossi)

# print(lossi)

In [ ]:
losses = []
lossi = []
vallosses = []
vallossi = []

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=4, drop_last=True, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=4, drop_last=True, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=4, drop_last=True, shuffle=False, num_workers=0)
#test_dataloader = DataLoader(eeg_dataset, batch_size=4, shuffle=False, num_workers=0)
print(len(train_dataloader), len(val_dataloader), len(test_dataloader))

15 5 5


In [ ]:
minValLoss = 9999

for epoch in tqdm(range(1)):
    for i, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        model.train()
        # print(window_tensors[i].shape)
        inputs = data['window']
        labels = data['labels']
        y_pred = model(inputs)
        # print(y_pred)

        # if epoch == 0:
        #     print(y_pred)

        loss = criterion(y_pred, labels)
        lossi.append(loss.item())
        loss.backward()
        optimizer.step()

        model.eval()
        for j, valdata in enumerate(val_dataloader):
            valinputs = valdata['window']
            vallabels = valdata['labels']
            val_pred = model(valinputs)
            loss = criterion(val_pred, vallabels)
            vallossi.append(loss.item())
        
        # if epoch <= 1:
        #     for p in model.parameters():
        #         print(p.grad)
    
    losses.append(np.mean(lossi))
    avgvalloss = np.mean(vallossi)
    if avgvalloss < minValLoss:
        minValLoss = avgvalloss
        torch.save(model.state_dict(), 'bestvalmodel.pt')
    vallosses.append(avgvalloss)
    if epoch % 100 == 0:
        plt.plot(losses, "b")
        plt.plot(vallosses, "g")
        plt.savefig('./valloss.png')
        plt.close()
    lossi = []
    vallossi = []
    

plt.plot(losses, "b")
plt.plot(vallosses, "g")

print(losses)

print(losses[-1])
torch.save(model.state_dict(), 'finaltrainmodel.pt')



  0%|          | 0/1 [00:00<?, ?it/s]/opt/miniconda3/envs/ml/lib/python3.10/site-packages/torch/nn/modules/instancenorm.py:115: UserWarning: input's size at dim=0 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(
  0%|          | 0/1 [00:00<?, ?it/s]

Shape check torch.Size([4, 1, 1250])
Shape check 2 torch.Size([4, 32, 1250])
Shape check 3 torch.Size([4, 32, 1250])
Shape check 4 torch.Size([4, 32, 416])
Shape check 5 torch.Size([4, 64, 416])
Shape check 6 torch.Size([4, 64, 138])
Shape check 7 torch.Size([4, 128, 138])
Shape check 8 torch.Size([4, 128, 46])
Shape check 9 torch.Size([4, 128, 1])
Shape check 10 torch.Size([4, 128])
Final shape torch.Size([4, 3])


ValueError: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4, 3])) is deprecated. Please ensure they have the same size.

In [ ]:
# minValLoss = 9999

# for epoch in tqdm(range(1)):
#     for i in range(len(window_tensors)):
#         optimizer.zero_grad()
#         model.train()
#         print(window_tensors[i].shape)
#         y_pred = model(window_tensors[i])
#         # print(y_pred)

#         # if epoch == 0:
#         #     print(y_pred)

#         loss = criterion(y_pred, label_tensors[i])
#         lossi.append(loss.item())
#         loss.backward()
#         optimizer.step()

#         model.eval()
#         for i in range(len(val_window_tensors)):
#             val_pred = model(val_window_tensors[i])
#             loss = criterion(val_pred, val_label_tensors[i])
#             vallossi.append(loss.item())
        
#         # if epoch <= 1:
#         #     for p in model.parameters():
#         #         print(p.grad)
    
#     losses.append(np.mean(lossi))
#     avgvalloss = np.mean(vallossi)
#     if avgvalloss < minValLoss:
#         minValLoss = avgvalloss
#         torch.save(model.state_dict(), 'bestvalmodel.pt')
#     vallosses.append(avgvalloss)
#     if epoch % 100 == 0:
#         plt.plot(losses, "b")
#         plt.plot(vallosses, "g")
#         plt.savefig('./valloss.png')
#         plt.close()
#     lossi = []
#     vallossi = []
    

# plt.plot(losses, "b")
# plt.plot(vallosses, "g")

# print(losses)

# print(losses[-1])
# torch.save(model.state_dict(), 'finaltrainmodel.pt')



  0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 1250])
Shape check torch.Size([1, 1250])
Shape check 2 torch.Size([32, 1250])


ValueError: expected 3D or 4D input (got 2D input)

In [ ]:
minindex = torch.argmin(torch.tensor(vallosses))
print(losses[minindex], vallosses[minindex])

0.5267679425518191 0.42550390597520144


In [ ]:
modelWeights = torch.load('./bestvalmodel.pt', weights_only=True)
#modelWeights = torch.load('./finaltrainmodel.pt', weights_only=True)
model = EEGEyesOpenCloseClassifier()
model.load_state_dict(modelWeights)
model.eval()
truepreds = 0
falsepreds = 0
truepositives = 0
falsepositives = 0
truenegatives = 0
falsenegatives = 0
preds = []
labels = []

with torch.no_grad():
    for i in range(len(test_dataset)):
        if i < 3:
            print(test_dataset[i]['window'])
        y_pred = model(test_dataset[i]['window'])
        # print(y_pred, test_dataset[i]['labels'])
        pred = (y_pred >= 0.5)
        preds.append(pred)
        label = test_dataset[i]['labels']
        labels.append(label)
        if pred == label:
            truepreds += 1
            if pred == 1:
                truepositives += 1
            else:
                truenegatives += 1
        else:
            falsepreds += 1
            if pred == 1:
                falsepositives += 1
            else:
                falsenegatives += 1

print(truepreds / (truepreds + falsepreds))
print(truepositives, truenegatives, falsepositives, falsenegatives)

f1Score = 2 * truepositives / (2 * truepositives + falsepositives + falsenegatives)
print(f1Score)

f1ScoreSKL = f1_score(labels, preds)
print(f1ScoreSKL)
        

tensor([[0.7357, 0.4009, 0.7700,  ..., 0.4009, 0.0419, 0.3947]])
tensor([[0.8350, 0.5764, 0.7985,  ..., 0.0136, 0.2617, 0.0000]])
tensor([[0.5309, 0.7825, 0.4786,  ..., 0.1769, 0.3101, 0.2576]])
0.85
9 8 3 0
0.8571428571428571
0.8571428571428571


In [ ]:
print((eeg_dataset[0]['window']))
#print(torch.min((no_normalize[0]['window'] - torch.min(no_normalize[0]['window']))/(torch.max(no_normalize[0]['window']))))

tensor([[0.7548, 0.5327, 0.8003,  ..., 0.4127, 0.7042, 0.4506]])


In [ ]:
# torch.save(model, 'oct7model.pt')